# TDSP Stage 1: Business Understanding

## Project: Time-Series Forecasting: Bitcoin (BTC) Price Prediction
**Domain:** Cryptocurrency / Financial Tech

### 1.1 Context
Bitcoin adalah aset digital dengan volatilitas tinggi yang dipengaruhi oleh sentimen pasar, adopsi teknologi, regulasi global, dan kondisi makroekonomi. Berbeda dengan emas yang cenderung stabil, pergerakan harga Bitcoin bisa sangat drastis dalam waktu singkat. Prediksi harga yang akurat penting untuk strategi *trading*, manajemen risiko portofolio, dan analisis tren jangka panjang.

### 1.2 Problem Statement
Stakeholder ingin memprediksi harga penutupan (*closing price*) Bitcoin bulanan di masa depan menggunakan data historis (2014-2025). Tujuannya adalah menguji efektivitas model *time-series* klasik (ARIMA, SARIMA) dan modern (Prophet) pada aset dengan volatilitas tinggi.

### 1.3 Goals
* Memproses data harian Bitcoin menjadi tren bulanan.
* Memprediksi harga Bitcoin untuk 24 bulan ke depan.
* Membandingkan akurasi model berdasarkan RMSE.

### 1.4 Analytic Approach
* **Data Transformation:** Resampling data harian menjadi rata-rata bulanan.
* **Metric:** RMSE (Root Mean Squared Error).